In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path('Resources/lending_data.csv')
lending_df =pd.read_csv(file_path)

# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
X = lending_df.drop(columns=['loan_status'])

In [4]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_reg_model = LogisticRegression(solver='lbfgs', random_state=1)

# # Fit the model using training data
logistic_reg_model.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
predictions = logistic_reg_model.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
60914,0,0
36843,0,0
1966,0,0
70137,0,0
27237,0,0
...,...,...
45639,0,0
11301,0,0
51614,0,0
4598,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.9918489475856377

In [11]:
# Generate a confusion matrix for the model
confusion_matrix(predictions,y_test)

array([[18663,    56],
       [  102,   563]])

In [12]:
# Print the classification report for the model
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     18719
           1       0.91      0.85      0.88       665

    accuracy                           0.99     19384
   macro avg       0.95      0.92      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model has a high level of accuracy with a score of .99. It appears that it does a better job at predicting healthy loans as opposed to unhealthy loans, as it has a precision of .99 and recall of 1.00 for healthy loans and .91 precision and .85 recall for unhealthy loans. That being said, when determining credit risk when extending loans, the determining factor is whether we can predict that the loan will be healthy before extending it. If a potential loan was falsely identified as being a risky loan, that potential business may be lost if we don't extend the loan, but nonetheless, potential risk is negated. 

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [20]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
rand_oversampler_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_res, y_res = rand_oversampler_model.fit_resample(X_train,y_train)

In [21]:
# Count the distinct values of the resampled labels data
y_res.value_counts()

loan_status
0    56271
1    56271
Name: count, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [23]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lrm_2 = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
lrm_2.fit(X_res, y_res)

# Make a prediction using the testing data
predictions = lrm_2.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
60914,0,0
36843,0,0
1966,0,0
70137,0,0
27237,0,0
...,...,...
45639,0,0
11301,0,0
51614,0,0
4598,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [24]:
# Print the balanced_accuracy score of the model 
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.9938093272802311

In [25]:
# Generate a confusion matrix for the model
confusion_matrix(predictions,y_test)

array([[18649,     4],
       [  116,   615]])

In [26]:
# Print the classification report for the model
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     18653
           1       0.99      0.84      0.91       731

    accuracy                           0.99     19384
   macro avg       0.99      0.92      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** After resampling the data, we see similar results with the regression model for healthy loans, as well as overall accuracy. However, for predicting unhealthy loans, we get a much better level of precision at .99 and a slightly lower recall at .85, which still leads to a higher f1-score of .91. This means that we're seeing slightly better performance in the model after oversampling the data. 